## part_B
* continuation of [part_A](part_A.ipynb)
* the graph is 9-colorable, it would be interesting to 
  know that 8 or less color is enough or not.
* for this purpose, we develop (actually i wrote it for a codesignal interview question...) a simple function in julia, that can be used to compute the chromatic number.

In [7]:
import Pkg; Pkg.activate(".")
Pkg.instantiate()
using DelimitedFiles
using Graphs, GraphPlot, Colors
using DataFrames
using StatsBase


  Activating project at `~/Asztal/git/notebooks/graph-coloring-with-networkx`


In [26]:
# reading and manipulating as in project_A:
d0,h0=readdlm(
  "synthetic_school_enrollment_data.csv",','; 
  header=true
)

# convert the original data

# drop out the first three columns (name,major/minor)
# and convert it to a valid logical matrix
data=map(
  x->if x=="True"
    true
  elseif x=="False"
    false
  else
    throw(error("unknown value"))
  end, 
  d0[:,4:end]
)

header=h0[4:end]
num_of_students,num_of_courses=size(data)

# build the graph:
# the nodes are the courses with an edge between them if there is a student visiting either.

# first, collect the set of students visiting each courses
S=[Set((1:num_of_students)[col]) for col in eachcol(data)]


# then, use the sets
G=Graph()
add_vertices!(G,num_of_courses)
for i in 1:num_of_courses-1, j in i+1:num_of_courses
  !isdisjoint(S[i],S[j]) && add_edge!(G,i,j)
end

G

{40, 273} undirected simple Int64 graph

In [67]:
# the backtracking solution
# it is a naive implementation w/o any smartness,
# just administration

function graphcol_bt(G::Vector{Vector{Int}},maxcol::Int) # max number of colors
# the actual color are in 1..maxcol
  N=length(G)
  
  forbidden=fill(0,maxcol,N) 
  # colors currently forbidden for a particular node
  # forbidden=already reserved by some of its neighbour
  # reserved if >0
  
  # actual and returned colorings
  color=fill(0,N) # for work with
  color_ret=fill(0,N)  

  # modifies the forbidden and color arrays
  function paint(node,c)
    oldc=color[node]
    if oldc>0
      for t in G[node]
        forbidden[oldc,t]-=1
      end
    end
    
    color[node]=c
    (c==0) && return
    
    for t in G[node]
      forbidden[c,t]+=1
    end
  end

  found=false
  paint(1,1)

  function trav(node)
    if node>N
      found=true
      color_ret.=color
      return
    end

    
    for c in 1:maxcol
      (forbidden[c,node]>0) && continue
      paint(node,c)
      trav(node+1)
      found && break
    end
    
    paint(node,0) # restore the original state
  end # of trav
  
  trav(2)
  (found,color_ret)
end


# a method (variant) that takes a Graph() instance and returns a similar
# object that is returned by Graphs.random_greedy_color
# (imitating by namedtuple)
function graphcol_bt(G::Graph,maxcol::Int) # max number of colors
  GG=[Int[] for n in 1:nv(G)]
  for e in edges(G)
    a,b=src(e),dst(e)
    push!(GG[a],b)
    push!(GG[b],a)    
  end
  found,color=graph_col_bt(GG,maxcol)
  if found
    (num_colors=length(Set(color)),colors=color)
  else
    (num_colors=-1,colors=nothing)
  end
end


graph_col_bt (generic function with 2 methods)

In [78]:
@time the_coloring=graphcol_bt(G,9);println(the_coloring)
@time failed=graphcol_bt(G,8);println(failed)


# on my machine it takes ~10 sec to compute the 9 coloring, 
# and ~4 sec to decide about the impossibility of the 8 coloring.

  9.547886 seconds (131 allocations: 22.523 KiB)
(num_colors = 9, colors = [1, 2, 3, 4, 5, 6, 7, 8, 2, 3, 1, 4, 7, 6, 5, 9, 8, 4, 1, 2, 3, 7, 5, 8, 9, 6, 1, 5, 4, 3, 8, 7, 9, 2, 4, 6, 7, 2, 1, 5])
  3.603913 seconds (124 allocations: 21.094 KiB)
(num_colors = -1, colors = nothing)


In [81]:
# we need maxcolsize rooms
mincolsize,maxcolsize=extrema(
  nc for (c,nc) in the_coloring.colors|>countmap
)


# build the final table
# exams for courses with the color 'k' will be held on the 'k'-th date given
table=fill("-",the_coloring.num_colors,maxcolsize) # indices for filling in
idx=fill(0,the_coloring.num_colors)
for i in 1:num_of_courses
  ri=the_coloring.colors[i]
  ci=(idx[ri]+=1)
  table[ri,ci]=header[i]
end

df=DataFrame(
  hcat("Exam-".*string.(1:the_coloring.num_colors),table),
  vcat("Exam","Room-".*string.(1:maxcolsize)))

  

Row,Exam,Room-1,Room-2,Room-3,Room-4,Room-5
,String,String,String,String,String,String
1,Exam-1,Biology of the Cell,Classical Mechanics,Calculus II,Programming Introduction,Tectonics
2,Exam-2,Molecular Biology,Quantum Mechanics,Probability I,Artificial Inteligence,Glaciology
3,Exam-3,Evolution,Thermodynamics,Probability II,Programming in C++,-
4,Exam-4,Biochemistry,Programming for Physics,Calculus I,Software Engineering,Ecology
5,Exam-5,Neurobiology,Material Science,Statistics II,Algorithms,Weather Systems
6,Exam-6,Animal Behavior,Complex Systems,Programming for Mathematics,Chemical Geology,-
7,Exam-7,Genetics,Linear Algebra for the Sciences,Statistics I,Data Science,Physical Geology
8,Exam-8,Bioinformatics,Robotics,Linear Algebra,Numerical Methods,-
9,Exam-9,Nanotechnologies,Geometry,Machine Learning,-,-


#### Conclusion
* for this graph instance *this* backtracking solution is very slow, 
  but after executing it we can be sure that fewer than 9 colors (exam dates) is not enough.